In [1]:
# 1) ENVIRONMENT SETUP (run in Colab)
# - Installs Java + PySpark + other Python libs
# - Recommended for Colab environment
import sys
import time
from IPython.display import clear_output, display
print("Starting environment setup...")

# Install system and Python packages - these commands run in the shell (Colab)
# Note: In Colab, prefix shell commands with "!" — the notebook will allow it.
# If you run this locally, adapt install commands appropriately.
!apt-get update -qq
!apt-get install -y -qq openjdk-11-jdk-headless
!pip install -q pyspark==3.4.1
!pip install -q requests beautifulsoup4 lxml matplotlib seaborn

print("Packages installed. Importing modules...")

Starting environment setup...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package openjdk-11-jre-headless:amd64.
(Reading database ... 121703 files and directories currently installed.)
Preparing to unpack .../openjdk-11-jre-headless_11.0.28+6-1ubuntu1~22.04.1_amd64.deb ...
Unpacking openjdk-11-jre-headless:amd64 (11.0.28+6-1ubuntu1~22.04.1) ...
Selecting previously unselected package openjdk-11-jdk-headless:amd64.
Preparing to unpack .../openjdk-11-jdk-headless_11.0.28+6-1ubuntu1~22.04.1_amd64.deb ...
Unpacking openjdk-11-jdk-headless:amd64 (11.0.28+6-1ubuntu1~22.04.1) ...
Setting up openjdk-11-jre-headless:amd64 (11.0.28+6-1ubuntu1~22.04.1) ...
update-alternatives: using /usr/lib/jvm/java-11-openjdk-amd64/bin/jjs to provide /usr/bin/jjs (jjs) in auto mode
update-alternatives: using /usr/lib/jvm/java-11-openjdk

In [2]:
# Standard imports (must be after pip installs when running in a fresh Colab runtime)
import os
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

In [3]:
 #2) START SPARK SESSION
# ---------------------------
# Create a SparkSession suitable for Colab usage
# Adjust appName / master as needed. This uses local mode.
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ColabNewsSentiment") \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
print("Spark session started:", spark)

Spark session started: <pyspark.sql.session.SparkSession object at 0x798dc1aff4d0>


In [4]:
# 3) NEWS FETCHING (Real-Time Simulation)
# ---------------------------
# Strategy:
# - Default: fetch latest headlines from a public RSS feed (no API key needed).
#   Example: BBC News RSS feed: http://feeds.bbci.co.uk/news/rss.xml
# - Alternate (commented) option: use NewsAPI (requires an API key) or other paid services.
#
# Where to put API key: if you choose to use NewsAPI, place your key in NEWSAPI_KEY variable
# (see commented example below).
#
# Implementation: fetch_headlines() -> returns list of headline strings

def fetch_headlines_from_rss(rss_url="http://feeds.bbci.co.uk/news/rss.xml", max_items=50):
    """
    Fetches headlines from an RSS feed URL (public, no key).
    Returns a Python list of headline strings.
    """
    try:
        resp = requests.get(rss_url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.content, "xml")
        items = soup.find_all("item")
        headlines = []
        for it in items[:max_items]:
            title_tag = it.find("title")
            if title_tag:
                headlines.append(title_tag.get_text().strip())
        return headlines
    except Exception as e:
        print("RSS fetch failed:", e)
        return []

# Example: how to use a real News API (commented)
# NEWSAPI_KEY = "<YOUR_NEWSAPI_KEY>"  # <-- put key here if you want to use NewsAPI
# def fetch_with_newsapi(query="world", page_size=50):
#     url = f"https://newsapi.org/v2/everything?q={query}&pageSize={page_size}&apiKey={NEWSAPI_KEY}"
#     r = requests.get(url)
#     data = r.json()
#     return [a['title'] for a in data.get('articles', [])]

# Initial fetch to seed headlines:
seed_headlines = fetch_headlines_from_rss(max_items=100)
print(f"Fetched {len(seed_headlines)} seed headlines (sample):")
for h in seed_headlines[:10]:
    print(" -", h)

# If no headlines were fetched (network issues), fallback to a small synthetic list:
if len(seed_headlines) == 0:
    print("No RSS headlines found - falling back to synthetic headlines for streaming.")
    seed_headlines = [
        "Government announces new economic measures to boost growth",
        "Local team wins dramatic overtime victory in championship",
        "Company reports mixed quarterly results as markets react",
        "Severe weather warnings issued across coastal regions",
        "Tech startup raises funds for AI-driven healthcare app",
        "Factory explosion injures several workers in industrial zone",
        "Scientists report breakthrough in renewable energy storage",
        "Protests erupt after controversial court ruling"
    ]

# ---------------------------


Fetched 38 seed headlines (sample):
 - Google boss warns 'no company is going to be immune' if AI bubble bursts
 - Don't blindly trust what AI tells you, says Google's Sundar Pichai
 - StubHub and Wayfair among firms named in online pricing investigation
 - Unprecedented plan for asylum system sees government walk tightrope
 - So long, plastic wet wipes - but should we be flushing the new ones?
 - KPop Demon Hunters star on how her life mirrored main character's journey
 - Paralegal sacked after offering to help dodge £60k illegal working fines
 - Dan Wootton denies High Court claim that he catfished 'former colleague'
 - Trump's plan for Gaza backed by UN Security Council
 - Shelters plea for Gazans as winter rains raise fears of more disease and death


In [5]:
# 4) SYNTHETIC TRAINING DATA (small dataset generated programmatically)
# ---------------------------
# We create a small dataset of labeled headlines (Positive/Negative) programmatically,
# using templates to produce a few hundred examples quickly and reproducibly.
# This keeps the notebook self-contained and runnable immediately.

positive_templates = [
    "growth", "wins", "soars", "improves", "record high", "surge", "positive outlook",
    "successful", "recovery", "gain", "reform", "approval", "solves", "breakthrough", "expand"
]
negative_templates = [
    "loss", "crash", "decline", "fails", "downturn", "scandal", "controversy", "protest",
    "injures", "dies", "arrested", "attack", "warning", "concern", "shortage"
]

def generate_synthetic_headlines(n=300):
    """
    Generate n synthetic (headline, label) tuples.
    Label is 'Positive' or 'Negative'.
    """
    samples = []
    for i in range(n):
        if random.random() < 0.5:
            # Positive
            t = random.choice(positive_templates)
            headline = f"{random.choice(['Company','Market','Team','Government','City'])} {t} as {random.choice(['demand rises','policy works','efforts pay off','confidence returns'])}"
            label = "Positive"
        else:
            # Negative
            t = random.choice(negative_templates)
            headline = f"{random.choice(['Company','Market','Region','Hospital','Agency'])} {t} after {random.choice(['incident','reports','investigation','shortage','accident'])}"
            label = "Negative"
        samples.append((headline, label))
    return samples

# Generate synthetic training data
synthetic_data = generate_synthetic_headlines(n=400)
print("Generated synthetic training dataset size:", len(synthetic_data))
print("Sample examples:")
for s in synthetic_data[:6]:
    print(" -", s)


Generated synthetic training dataset size: 400
Sample examples:
 - ('Government approval as efforts pay off', 'Positive')
 - ('Government expand as demand rises', 'Positive')
 - ('Region arrested after accident', 'Negative')
 - ('Region warning after reports', 'Negative')
 - ('Team breakthrough as confidence returns', 'Positive')
 - ('City breakthrough as policy works', 'Positive')


In [6]:
# 5) CREATE TRAINING SPARK DATAFRAME AND TRAIN PySpark ML PIPELINE
# ---------------------------
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create Spark DataFrame from synthetic_data
rows = [Row(text=txt, label=lab) for txt, lab in synthetic_data]
train_df = spark.createDataFrame(rows)
train_df = train_df.select(col("text"), col("label"))

# Convert string labels ("Positive"/"Negative") into numeric index for ML algorithms
label_indexer = StringIndexer(inputCol="label", outputCol="labelIndex")

# Build a pipeline: Tokenizer -> StopWordsRemover -> HashingTF -> IDF -> LogisticRegression
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
stop_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
hashing_tf = HashingTF(inputCol="filtered_tokens", outputCol="rawFeatures", numFeatures=1 << 12)  # 4096
idf = IDF(inputCol="rawFeatures", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="labelIndex", maxIter=50)

pipeline = Pipeline(stages=[label_indexer, tokenizer, stop_remover, hashing_tf, idf, lr])

# Fit the pipeline
print("Training pipeline on synthetic data (this may take a few seconds)...")
pipeline_model = pipeline.fit(train_df)
print("Pipeline trained.")

# Evaluate quickly on training data (sanity-check)
preds = pipeline_model.transform(train_df)
evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
acc = evaluator.evaluate(preds)
print(f"Training accuracy (synthetic data): {acc:.3f}")

# Map numerical predictions back to string labels:
# We need to know how StringIndexer mapped "Positive"/"Negative" -> numeric
label_mapping = pipeline_model.stages[0].labels  # list like ['Negative','Positive'] or vice versa
print("Label mapping (index -> label):", list(enumerate(label_mapping)))

def index_to_label(idx):
    # helper: convert numeric index to label string
    return label_mapping[int(idx)]

Training pipeline on synthetic data (this may take a few seconds)...
Pipeline trained.
Training accuracy (synthetic data): 1.000
Label mapping (index -> label): [(0, 'Positive'), (1, 'Negative')]


In [7]:
# 6) STREAMING SIMULATION: readStream('rate') + foreachBatch to apply pipeline_model
# ---------------------------
# Approach:
# - Use Spark Structured Streaming 'rate' source to generate sequential 'value' numbers.
# - In foreachBatch (which runs on driver), fetch latest headlines (using RSS), map value -> headline
# - Create a small batch DataFrame with columns 'text' and process it with pipeline_model.transform (batch-mode)
# - Aggregate results into a driver-side list for visualization
#
# Note: We intentionally use foreachBatch to keep model application simple and robust with PySpark ML.

from pyspark.sql.functions import expr

# Global store updated within foreachBatch; driver-only list for visualization
# Each element will be a dict with counts and timestamp
results_store = []  # driver-side list to store results per micro-batch

# Size of window for mapping rate.value -> headline index
headline_pool = seed_headlines.copy()  # initial seed headlines; foreachBatch will optionally refresh
headline_pool_len = len(headline_pool)
print("Using headline pool size:", headline_pool_len)

def foreach_batch_function(batch_df, batch_id):
    """
    This function will be called for each micro-batch by Structured Streaming.
    It runs on the driver; we:
      - fetch latest headlines (optional)
      - produce a list of headlines mapped from the 'value' column
      - create a Spark DataFrame of headlines and apply the trained pipeline_model
      - aggregate counts of Positive/Negative and append to results_store
    """
    global results_store, headline_pool, headline_pool_len

    # Convert micro-batch to Pandas to get 'value' list (small micro-batches expected)
    try:
        values = [int(r.value) for r in batch_df.select("value").collect()]
    except Exception as e:
        print("Error collecting batch values:", e)
        return

    if len(values) == 0:
        # nothing in this micro-batch
        return

    # Optionally fetch latest headlines from RSS on each batch to simulate 'real-time' new headlines
    try:
        fresh = fetch_headlines_from_rss(max_items=100)
        if len(fresh) > 0:
            headline_pool = fresh
            headline_pool_len = len(headline_pool)
    except Exception as e:
        # ignore fetch errors, keep existing pool
        pass

    # Map rate 'value's to headlines via modulo indexing
    batch_headlines = []
    for v in values:
        idx = int(v % headline_pool_len)
        batch_headlines.append((str(headline_pool[idx]),))  # single-column tuple

    # Create batch DataFrame with column 'text'
    batch_rows = [Row(text=h[0]) for h in batch_headlines]
    if len(batch_rows) == 0:
        return
    micro_df = spark.createDataFrame(batch_rows)

    # If pipeline_model expects a label column (it does during training) it's okay:
    # model.transform will produce predictions based on features.
    transformed = pipeline_model.transform(micro_df)

    # Collect predictions and compute counts
    pred_rows = transformed.select("text", "prediction").collect()
    pos_count = 0
    neg_count = 0
    details = []
    for r in pred_rows:
        label = index_to_label(r.prediction)
        details.append((r.text, label))
        if label == "Positive":
            pos_count += 1
        else:
            neg_count += 1

    # Append aggregated results (timestamp, counts, details) to results_store
    batch_time = time.time()
    results_store.append({
        "batch_id": int(batch_id),
        "ts": batch_time,
        "positive": pos_count,
        "negative": neg_count,
        "total": pos_count + neg_count,
        "details": details
    })

    # For debug: print small summary
    print(f"[batch {batch_id}] +{pos_count} pos / {neg_count} neg (total {pos_count + neg_count})")

# Create a streaming DataFrame using 'rate' to produce rows at runtime
# Each row has schema: timestamp, value
streaming_input = spark.readStream.format("rate").option("rowsPerSecond", 2).load()

# Start streaming query with foreachBatch sink
query = streaming_input.writeStream.foreachBatch(foreach_batch_function).start()
print("Structured streaming query started. Query id:", query.id)


Using headline pool size: 38
Structured streaming query started. Query id: f253b319-0295-47bb-a16a-d51a6b3750a6


In [8]:
# 7) REAL-TIME DASHBOARD (driver-side)
# ---------------------------
# We'll run a short live visualization loop that polls results_store and plots Positive vs Negative.
# The streaming query is running in background; foreachBatch will append to results_store regularly.

print("Starting live dashboard (will poll for 30 seconds). Press interrupt to stop sooner.")

display_interval = 1.0  # seconds between dashboard refresh
dashboard_duration = 30  # seconds for demonstration (adjust as desired)

start_time = time.time()
try:
    while time.time() - start_time < dashboard_duration:
        # Build aggregated counts from results_store
        total_pos = sum(item['positive'] for item in results_store)
        total_neg = sum(item['negative'] for item in results_store)
        total = total_pos + total_neg

        clear_output(wait=True)
        print(f"Real-time News Sentiment Dashboard (simulated) — running for {int(time.time()-start_time)}s")
        print(f"Total processed headlines: {total}    Positive: {total_pos}    Negative: {total_neg}\n")

        # Plot counts as a bar chart
        fig, ax = plt.subplots(figsize=(6,3))
        sns.barplot(x=["Positive", "Negative"], y=[total_pos, total_neg], ax=ax)
        ax.set_ylim(0, max(5, total_pos + total_neg))
        ax.set_title("Cumulative Sentiment Counts (simulated real-time)")
        for i, v in enumerate([total_pos, total_neg]):
            ax.text(i, v + 0.05, str(v), ha="center")
        plt.show()

        # Print last batch details (if present)
        if len(results_store) > 0:
            last = results_store[-1]
            print(f"Last batch (id={last['batch_id']}) processed at {time.ctime(last['ts'])}: "
                  f"{last['positive']} positive, {last['negative']} negative")
            print("Sample classifications from last batch:")
            for sample_text, sample_label in last["details"][:5]:
                print(" -", sample_label, ":", sample_text)

        time.sleep(display_interval)

except KeyboardInterrupt:
    print("Dashboard interrupted by user.")


Real-time News Sentiment Dashboard (simulated) — running for 11s
Total processed headlines: 24    Positive: 2    Negative: 22

Dashboard interrupted by user.


In [ ]:
# Spark ML provides its own methods for saving and loading models
pipeline_model.save("sentiment_pipeline_model")

print("✅ Pipeline model saved successfully!")

[batch 1491] +0 pos / 2 neg (total 2)
[batch 1492] +0 pos / 2 neg (total 2)
[batch 1493] +0 pos / 2 neg (total 2)
[batch 1494] +1 pos / 1 neg (total 2)
[batch 1495] +1 pos / 1 neg (total 2)
[batch 1496] +0 pos / 2 neg (total 2)
✅ Pipeline model saved successfully!


In [9]:
# Install dependencies
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Create the analyzer
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(news_text):
    scores = analyzer.polarity_scores(news_text)
    compound = scores["compound"]

    if compound >= 0.05:
        return "Positive", scores
    elif compound <= -0.05:
        return "Negative", scores
    else:
        return "Neutral", scores


# Test
while True:
    news = input("\nEnter a news headline (or type 'exit'): ")
    if news.lower() == "exit":
        break

    sentiment, score = get_sentiment(news)
    print(f"\nSentiment: {sentiment}")
    print(f"Scores: {score}")


[batch 19] +0 pos / 2 neg (total 2)
[batch 20] +0 pos / 2 neg (total 2)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.5 MB/s eta 0:00:00
[batch 21] +0 pos / 2 neg (total 2)
[batch 22] +0 pos / 2 neg (total 2)
[batch 23] +1 pos / 3 neg (total 4)
[batch 24] +0 pos / 2 neg (total 2)
[batch 25] +0 pos / 2 neg (total 2)
[batch 26] +0 pos / 2 neg (total 2)
[batch 27] +0 pos / 2 neg (total 2)
[batch 28] +0 pos / 2 neg (total 2)
[batch 29] +0 pos / 4 neg (total 4)
[batch 30] +0 pos / 2 neg (total 2)
[batch 31] +0 pos / 2 neg (total 2)
[batch 32] +0 pos / 2 neg (total 2)
[batch 33] +0 pos / 2 neg (total 2)
[batch 34] +1 pos / 1 neg (total 2)
[batch 35] +0 pos / 2 neg (total 2)
[batch 36] +0 pos / 2 neg (total 2)
[batch 37] +0 pos / 2 neg (total 2)
[batch 38] +0 pos / 2 neg (total 2)
[batch 39] +0 pos / 2 neg (total 2)
[batch 40] +0 pos / 2 neg (total 2)
[batch 41] +1 pos / 1 neg (total 2)
[batch 42] +0 pos / 2 neg (total 2)
[batch 43] +0 pos / 2 neg (total 2)
[batch 44] +0 pos / 